# Transformaciones producto
En este notebook se harán las transformaciones que se discuten en el notebook de exploración de datasets respecto a `Producto.parquet`.

In [1]:
import pandas as pd
import unidecode
import numpy as np

In [2]:
prod = pd.read_parquet('../datasets/producto.parquet')
prod.head()

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un,None,None,None
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un,None,None,None
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg,None,None,None
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg,None,None,None
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg,None,None,None


In [3]:
# ids unicos
prod.id.unique().shape

(72038,)

In [4]:
# espacios en blanco y primera letra mayuscula
prod.marca = prod.marca.str.strip().str.capitalize()
prod.head()

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,La anónima,Radicheta Atada La Anonima 1 Un,1.0 un,None,None,None
1,0000000002288,La anónima,Perejil Atado La Anonima 1 Un,1.0 un,None,None,None
2,0000000205870,Sin marca,Ojo de Bife 1 Kg,1.0 kg,None,None,None
3,0000000205894,Sin marca,Milanesa de Peceto Novillito 1 Kg,1.0 kg,None,None,None
4,0000000205955,Sin marca,Chiquizuela Novillito 1 Kg,1.0 kg,None,None,None


In [5]:
# Eliminamos la presentacion del nombre
prod.nombre = prod.nombre.str.split('(\d+)', expand=True)[0]
# Espacios en blanco y letra mayuscula
prod.nombre = prod.nombre.str.strip().str.capitalize()
prod.head()

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un,None,None,None
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un,None,None,None
2,0000000205870,Sin marca,Ojo de bife,1.0 kg,None,None,None
3,0000000205894,Sin marca,Milanesa de peceto novillito,1.0 kg,None,None,None
4,0000000205955,Sin marca,Chiquizuela novillito,1.0 kg,None,None,None


In [6]:
# dropeamos columnas inutiles
prod_error_aux = prod.loc[prod.categoria1.notnull()].copy()
prod_error_aux['tipoError'] = 0
# cambiamos el nombre de id
prod_error_aux.rename({'id':'antiguoId'}, axis=1, inplace=True)
prod_error_aux.head()

,antiguoId,marca,nombre,presentacion,categoria1,categoria2,categoria3,tipoError
14903,7501001165260,Pantene,Acondicionador liso y sedoso pantene,400.0 ml,Perfumería y Cuidado Personal,Cuidado Capilar,Acondicionadores,0
53619,7798024299045,None,None,None,Almacén,Aceites,Aerosol,0
55798,7798061190213,None,None,None,Almacén,Aceites,Oliva,0
65297,7798183771123,Grido,Helado en pote tentacion,1.0 lt,Alimentos Congelados,Helados,Helados,0


In [7]:
# columnas
prod.drop(['categoria1','categoria2','categoria3'],axis='columns', inplace=True)
prod.head()

# filas (coinciden con las filas que se copiaron a aux)
prod.drop(prod.loc[(prod.nombre.isnull()) | (prod.marca.isnull())].index, axis='index', inplace=True)

prod.head()

,id,marca,nombre,presentacion
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un
2,0000000205870,Sin marca,Ojo de bife,1.0 kg
3,0000000205894,Sin marca,Milanesa de peceto novillito,1.0 kg
4,0000000205955,Sin marca,Chiquizuela novillito,1.0 kg


In [8]:
# tipos de unidades que hay
prod.presentacion.str.split(expand=True)[1].unique()

# puedo disminuir la cantidad de unidades
# 1000 ml =  1000 cc = 1 lt
# 1000 gr = 1 kg 

array(['un', 'kg', 'gr', 'ml', 'cc', 'lt', 'mt'], dtype=object)

In [9]:
# Hacemos split y creamos una columna para cantidad y unidad
prod['cantidad'] = prod.presentacion.str.split(expand=True)[0].astype(float)
prod['unidad'] = prod.presentacion.str.split(expand=True)[1]
prod.head()

,id,marca,nombre,presentacion,cantidad,unidad
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un,1.0,un
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un,1.0,un
2,0000000205870,Sin marca,Ojo de bife,1.0 kg,1.0,kg
3,0000000205894,Sin marca,Milanesa de peceto novillito,1.0 kg,1.0,kg
4,0000000205955,Sin marca,Chiquizuela novillito,1.0 kg,1.0,kg


In [10]:
prod.loc[prod.unidad == 'kg', 'cantidad'] = prod.cantidad * 1000
prod.loc[prod.unidad == 'kg', 'unidad'] = 'gr'

prod.loc[prod.unidad == 'lt', 'cantidad'] = prod.cantidad * 1000
prod.loc[prod.unidad == 'lt', 'unidad'] = 'ml'
prod.loc[prod.unidad == 'cc', 'unidad'] = 'ml'

# Tenemos menos unidades
prod.unidad.unique()

array(['un', 'gr', 'ml', 'mt'], dtype=object)

In [11]:
prod

,id,marca,nombre,presentacion,cantidad,unidad
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un,1.0,un
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un,1.0,un
2,0000000205870,Sin marca,Ojo de bife,1.0 kg,1000.0,gr
3,0000000205894,Sin marca,Milanesa de peceto novillito,1.0 kg,1000.0,gr
4,0000000205955,Sin marca,Chiquizuela novillito,1.0 kg,1000.0,gr
...,...,...,...,...,...,...
72033,9569753142128,Deli-sitas,Milhojas cobertura de chocolate blanco deli-sitas,500.0 gr,500.0,gr
72034,9795403001143,Mayo,Mini pizzetas mayo,12.0 un,12.0,un
72035,9990385651922,Dana,Te negro en hebras lata dana,50.0 gr,50.0,gr
72036,9990385651939,Dana,Te verde en hebras lata dana,50.0 gr,50.0,gr


In [12]:
# juntamos cantidad y unidad y sustituimos presentacion
prod['presentacion'] = prod.cantidad.astype(str) + ' ' + prod.unidad
prod.drop(['cantidad','unidad'], axis='columns', inplace=True)
prod.head()

,id,marca,nombre,presentacion
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un
2,0000000205870,Sin marca,Ojo de bife,1000.0 gr
3,0000000205894,Sin marca,Milanesa de peceto novillito,1000.0 gr
4,0000000205955,Sin marca,Chiquizuela novillito,1000.0 gr


In [13]:
prod

,id,marca,nombre,presentacion
0,0000000001663,La anónima,Radicheta atada la anonima,1.0 un
1,0000000002288,La anónima,Perejil atado la anonima,1.0 un
2,0000000205870,Sin marca,Ojo de bife,1000.0 gr
3,0000000205894,Sin marca,Milanesa de peceto novillito,1000.0 gr
4,0000000205955,Sin marca,Chiquizuela novillito,1000.0 gr
...,...,...,...,...
72033,9569753142128,Deli-sitas,Milhojas cobertura de chocolate blanco deli-sitas,500.0 gr
72034,9795403001143,Mayo,Mini pizzetas mayo,12.0 un
72035,9990385651922,Dana,Te negro en hebras lata dana,50.0 gr
72036,9990385651939,Dana,Te verde en hebras lata dana,50.0 gr


In [14]:
# Quitamos la marca del nombre
prod['marca_st'] = prod.apply((lambda row : unidecode.unidecode(str(row['marca']).lower())), axis=1)
prod.nombre = prod.apply(lambda row: row['nombre'].replace(str(row['marca_st']),''), axis=1)

# Dropeamos la columna auxiliar
prod.drop('marca_st', axis='columns', inplace=True)
prod.head()

,id,marca,nombre,presentacion
0,0000000001663,La anónima,Radicheta atada,1.0 un
1,0000000002288,La anónima,Perejil atado,1.0 un
2,0000000205870,Sin marca,Ojo de bife,1000.0 gr
3,0000000205894,Sin marca,Milanesa de peceto novillito,1000.0 gr
4,0000000205955,Sin marca,Chiquizuela novillito,1000.0 gr


In [15]:
# Para normalizar, hacemos una tabla de marcas
df_marca = prod.marca.to_frame().copy()
df_marca.drop_duplicates(inplace=True)
df_marca.insert(0, 'marcaId', range(1, 1 + len(df_marca)))
df_marca

,marcaId,marca
0,1,La anónima
2,2,Sin marca
24,3,Paladini
25,4,La paulina
26,5,Dulcor
...,...,...
70944,5561,La hacieda bandeja
71193,5562,Nexcare
72027,5563,Pez
72030,5564,Dilmah


In [16]:
# Mapping a producto
prod = pd.merge(prod, df_marca, on=['marca'])
prod.drop('marca', axis=True, inplace=True)
prod.reset_index(drop=True, inplace=True)

prod.head()

,id,nombre,presentacion,marcaId
0,0000000001663,Radicheta atada,1.0 un,1
1,0000000002288,Perejil atado,1.0 un,1
2,2000957000007,Piña,1000.0 gr,1
3,2001271000001,Carre de cerdo con hueso,1000.0 gr,1
4,2001273000009,Pechito con hueso de cerdo,1000.0 gr,1


In [17]:
# Vamos a convertir esta tabla en la tabla aux y crear una de productos
# donde los productos no se repitan
prod_aux = prod.copy()

# empezamos con los id
# son EAN, deberian ser de 13 digitos
# el otro formato de EAN que hallé (por ejemplo): 1-1-0000000205955
# nos quedamos con la ultima parte despues del segundo -
prod['id'].astype(str)
prod['codigoEAN'] = prod.loc[:, 'id']
prod

,id,nombre,presentacion,marcaId,codigoEAN
0,0000000001663,Radicheta atada,1.0 un,1,0000000001663
1,0000000002288,Perejil atado,1.0 un,1,0000000002288
2,2000957000007,Piña,1000.0 gr,1,2000957000007
3,2001271000001,Carre de cerdo con hueso,1000.0 gr,1,2001271000001
4,2001273000009,Pechito con hueso de cerdo,1000.0 gr,1,2001273000009
...,...,...,...,...,...
72031,9312631143560,Te en saquitos variety frutal,20.0 un,5564,9312631143560
72032,9312631144451,Te green en saquitos variety pack,20.0 un,5564,9312631144451
72033,9990385651922,Te negro en hebras lata,50.0 gr,5565,9990385651922
72034,9990385651939,Te verde en hebras lata,50.0 gr,5565,9990385651939


In [18]:
# formateamos para que tengan codigos EAN de 13 digitos
prod['codigoEAN'].loc[prod['codigoEAN'].str.len() != 13] = prod['codigoEAN'].loc[prod['codigoEAN'].str.len() != 13].str.split('-').str[2]

# chequeamos los id distintos
prod.loc[prod['codigoEAN'].str.len() != 13]

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,id,nombre,presentacion,marcaId,codigoEAN


In [19]:
prod.shape

(72036, 5)

In [20]:
# dropeamos las filas que son del mismo producto
prod.drop_duplicates(subset=['nombre','presentacion','marcaId','codigoEAN'], inplace=True)
prod.shape

(68186, 5)

In [21]:
# reseteamos indices y agregamos un id
prod.reset_index(inplace=True, drop=True)
prod['productoId'] = prod.index + 1 
prod.productoId.astype(int)
prod = prod[['productoId','nombre','presentacion','marcaId','codigoEAN','id']]
prod

,productoId,nombre,presentacion,marcaId,codigoEAN,id
0,1,Radicheta atada,1.0 un,1,0000000001663,0000000001663
1,2,Perejil atado,1.0 un,1,0000000002288,0000000002288
2,3,Piña,1000.0 gr,1,2000957000007,2000957000007
3,4,Carre de cerdo con hueso,1000.0 gr,1,2001271000001,2001271000001
4,5,Pechito con hueso de cerdo,1000.0 gr,1,2001273000009,2001273000009
...,...,...,...,...,...,...
68181,68182,Te en saquitos variety frutal,20.0 un,5564,9312631143560,9312631143560
68182,68183,Te green en saquitos variety pack,20.0 un,5564,9312631144451,9312631144451
68183,68184,Te negro en hebras lata,50.0 gr,5565,9990385651922,9990385651922
68184,68185,Te verde en hebras lata,50.0 gr,5565,9990385651939,9990385651939


In [22]:
# mapeamos el id a prod aux, para poder eliminar id de prod
prod_aux = pd.merge(prod, prod_aux, on=['nombre', 'presentacion', 'marcaId'], how = 'right')
prod_aux.drop(['id_x','nombre','presentacion','marcaId','codigoEAN'], axis='columns', inplace = True)
prod_aux.rename({'id_y':'antiguoId'}, axis=1, inplace=True)
prod_aux

,productoId,antiguoId
0,1,0000000001663
1,2,0000000002288
2,3,2000957000007
3,4,2001271000001
4,5,2001273000009
...,...,...
113516,68182,9312631143560
113517,68183,9312631144451
113518,68184,9990385651922
113519,68185,9990385651939


In [23]:
# Eliminamos id de la tabla de producto del modelo
prod.drop('id', axis='columns', inplace=True)
prod

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,productoId,nombre,presentacion,marcaId,codigoEAN
0,1,Radicheta atada,1.0 un,1,0000000001663
1,2,Perejil atado,1.0 un,1,0000000002288
2,3,Piña,1000.0 gr,1,2000957000007
3,4,Carre de cerdo con hueso,1000.0 gr,1,2001271000001
4,5,Pechito con hueso de cerdo,1000.0 gr,1,2001273000009
...,...,...,...,...,...
68181,68182,Te en saquitos variety frutal,20.0 un,5564,9312631143560
68182,68183,Te green en saquitos variety pack,20.0 un,5564,9312631144451
68183,68184,Te negro en hebras lata,50.0 gr,5565,9990385651922
68184,68185,Te verde en hebras lata,50.0 gr,5565,9990385651939


In [24]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68186 entries, 0 to 68185
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   productoId    68186 non-null  int64 
 1   nombre        68186 non-null  object
 2   presentacion  68186 non-null  object
 3   marcaId       68186 non-null  int64 
 4   codigoEAN     68186 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.6+ MB


In [25]:
# Guardamos los archivos producto de la limpieza
dfs = [prod, df_marca, prod_aux, prod_error_aux]
nombres = ['producto', 'producto_marca', 'producto_auxiliar', 'producto_error']

for df, nombre in zip(dfs, nombres):
    df.to_csv('../datasets/{}.csv'.format(nombre), index=False)